## Loading Data

This loads the Rotterdam data from my website.


In [1]:
. use https://www.pclambert.net/data/rott3, clear

(Rotterdam breast cancer data (augmented with cause of death))


## Fitting an `stpm3` model

We can now fit an flexible parametric model using `stpm3`.
I will include the variabe `hormon` as a factor variable 
and `age` as a natural cubic spline.


In [2]:
*| classes: styled-output
*|echo: false
%%echo
stpm3 i.hormon @ns(age,df(3)), scale(lncumhazard) df(4) initmod(exp) nolog
estimates store stpm3


. stpm3 i.hormon @ns(age,df(3)), scale(lncumhazard) df(4) initmod(exp) nolog



                                                        Number of obs =  2,982
                                                        Wald chi2(4)  = 132.42
Log likelihood = -2884.0429                             Prob > chi2   = 0.0000

------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
xb           |
      hormon |
        yes  |   .3731572   .0885452     4.21   0.000     .1996119    .5467026
 _ns_f1_age1 |  -3.142327   .8938032    -3.52   0.000    -4.894149   -1.390505
 _ns_f1_age2 |  -.9788548   .3796951    -2.58   0.010    -1.723043   -.2346661
 _ns_f1_age3 |  -2.199723   .3953551    -5.56   0.000    -2.974604   -1.424841
-------------+----------------------------------------------------------------
time         |
        _ns1 |  -25.52175   1.864219   -13.69   0.000    -29.17555   -21.86795
     

# Running `mlad'

We need some starting values fo the parameters

## Starting values

This is a simple way to get initial valuesb by fitting an exponential model,
predicting the log cumulative hazard anduse least squares.


In [3]:
*| classes: styled-output
*| code-fold: true
*| code-summary: "Show the code"
%%echo 
streg i.hormon _ns_f1_age1 _ns_f1_age2 _ns_f1_age3, dist(exp)
predict surv, surv
gen lncumH = ln(-ln(surv))
regress lncumH i.hormon _ns_f1_age1 _ns_f1_age2 _ns_f1_age3 _ns1 _ns2 _ns3 _ns4 if _d

// Store inital values
matrix b_init = e(b)


. streg i.hormon _ns_f1_age1 _ns_f1_age2 _ns_f1_age3, dist(exp)

         Failure _d: osi==1
   Analysis time _t: os/12
  Exit on or before: time 10*12
        ID variable: pid

Iteration 0:   log likelihood = -3022.1979  
Iteration 1:   log likelihood = -2977.5856  
Iteration 2:   log likelihood =  -2967.277  
Iteration 3:   log likelihood = -2967.2473  
Iteration 4:   log likelihood = -2967.2473  

Exponential PH regression

No. of subjects =       2,982                           Number of obs =  2,982
No. of failures =       1,171
Time at risk    = 20,002.4244
                                                        LR chi2(4)    = 109.90
Log likelihood = -2967.2473                             Prob > chi2   = 0.0000

------------------------------------------------------------------------------
          _t | Haz. ratio   Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
      hormon |
        yes  |   1


. gen lncumH = ln(-ln(surv))

. regress lncumH i.hormon _ns_f1_age1 _ns_f1_age2 _ns_f1_age3 _ns1 _ns2 _ns3 _n
> s4 if _d

      Source |       SS           df       MS      Number of obs   =     1,171
-------------+----------------------------------   F(8, 1162)      >  99999.00
       Model |  639.715523         8  79.9644404   Prob > F        =    0.0000
    Residual |  2.9834e-11     1,162  2.5675e-14   R-squared       =    1.0000
-------------+----------------------------------   Adj R-squared   =    1.0000
       Total |  639.715523     1,170  .546765404   Root MSE        =    1.6e-07

------------------------------------------------------------------------------
      lncumH | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
      hormon |
        yes  |   .3538171   1.41e-08  2.5e+07   0.000     .3538171    .3538172
 _ns_f1_age1 |  -2.976691   1.40e-07 -2.1e+07   0.000    -2.976691    